## Task 4: Building predictive model

- Objective: to predict claim severity prediction. Done to predict the Total Claim amount. 

In [1]:
import os
import sys

system_path = os.path.abspath('..')
if system_path not in sys.path:
    sys.path.append(system_path)

%load_ext autoreload
%autoreload 2

In [2]:
from src.predictive_modeling import InsuranceRiskModel
from src.eda_analysis import InsuranceEDA
eda = InsuranceEDA()
modeling = InsuranceRiskModel()

In [4]:
data = eda.load_data('../data/insurance_data.parquet')
data

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,31520,389,2015-04-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0
1000094,31520,389,2015-06-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0
1000095,31520,389,2015-08-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0
1000096,31519,389,2014-07-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Passenger Liability,Passenger Liability,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,2.315000,0.0


In [5]:
eda.check_missing_values()


--------------------------------Missing Values Analysis--------------------------------
Total number of columns with missing values: 22

Missing values summary:


,Missing Count,Missing Percentage
NumberOfVehiclesInFleet,1000098,100.000000
CrossBorder,999400,99.930207
CustomValueEstimate,779642,77.956560
Rebuilt,641901,64.183810
Converted,641901,64.183810
WrittenOff,641901,64.183810
NewVehicle,153295,15.327998
Bank,145961,14.594670
AccountType,40232,4.022806
Gender,9536,0.953507


In [7]:
# Remove columns with more than 5% missing values
missing_threshold = 5.0
missing_summary = eda.check_missing_values()

# Get columns with more than 5% missing values
columns_to_remove = missing_summary[missing_summary['Missing Percentage'] > missing_threshold].index.tolist()

print(f"Columns to remove (missing > {missing_threshold}%):")
for col in columns_to_remove:
    print(f"- {col}")

# Remove the columns from the dataset
data_cleaned = data.drop(columns=columns_to_remove)

print(f"\nOriginal dataset shape: {data.shape}")
print(f"Cleaned dataset shape: {data_cleaned.shape}")
print(f"Removed {len(columns_to_remove)} columns")

# Update the data in the EDA object
eda.data = data_cleaned



--------------------------------Missing Values Analysis--------------------------------
Total number of columns with missing values: 14

Missing values summary:
Columns to remove (missing > 5.0%):

Original dataset shape: (1000098, 52)
Cleaned dataset shape: (1000098, 52)
Removed 0 columns


In [14]:
# Create a subset of data where claims > 0 for modeling
claims_data = data_cleaned[data_cleaned['TotalClaims'] > 0].copy()
claims_data.to_parquet('../data/claims_data.parquet')
print(f"shape of claims_data: {claims_data.shape}")

shape of claims_data: (2788, 44)


In [23]:
claims_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2788 entries, 203 to 999923
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   UnderwrittenCoverID       2788 non-null   int64         
 1   PolicyID                  2788 non-null   int64         
 2   TransactionMonth          2788 non-null   datetime64[ns]
 3   IsVATRegistered           2788 non-null   bool          
 4   Citizenship               2788 non-null   object        
 5   LegalType                 2788 non-null   object        
 6   Title                     2788 non-null   object        
 7   Language                  2788 non-null   object        
 8   AccountType               2710 non-null   object        
 9   MaritalStatus             2773 non-null   object        
 10  Gender                    2774 non-null   object        
 11  Country                   2788 non-null   object        
 12  Province             

Data preparation 

In [16]:
preprocessed_data = modeling.prepare_data(claims_data, 'TotalClaims', test_size=0.2, random_state=42, task_type='classification')

Preparing data for classification task...
Original dataset shape: (2788, 44)
Engineering features...
Feature engineering complete. New shape: (2788, 53)
Handling missing values...
Missing values handled.
Numeric features: 18
Categorical features: 29
Training set shape: (2230, 52)
Test set shape: (558, 52)


In [17]:
modeling.build_models()

Built 3 models for regression task


In [22]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2230 entries, 528925 to 654314
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   UnderwrittenCoverID       2230 non-null   int64         
 1   PolicyID                  2230 non-null   int64         
 2   TransactionMonth          2230 non-null   datetime64[ns]
 3   IsVATRegistered           2230 non-null   bool          
 4   Citizenship               2230 non-null   object        
 5   LegalType                 2230 non-null   object        
 6   Title                     2230 non-null   object        
 7   Language                  2230 non-null   object        
 8   AccountType               2230 non-null   object        
 9   MaritalStatus             2230 non-null   object        
 10  Gender                    2230 non-null   object        
 11  Country                   2230 non-null   object        
 12  Province          

In [ ]:
# Convert TransactionMonth from datetime to string format
X_train['TransactionMonth'] = X_train['TransactionMonth'].dt.strftime('%Y-%m-%d')
X_test['TransactionMonth'] = X_test['TransactionMonth'].dt.strftime('%Y-%m-%d')

# Alternative: Convert to numeric representation (days since epoch)
# X_train['TransactionMonth'] = X_train['TransactionMonth'].dt.to_numpy().astype(np.int64) // 10**9
# X_test['TransactionMonth'] = X_test['TransactionMonth'].dt.to_numpy().astype(np.int64) // 10**9

print("TransactionMonth converted to string format")
print(f"X_train TransactionMonth dtype: {X_train['TransactionMonth'].dtype}")
print(f"X_test TransactionMonth dtype: {X_test['TransactionMonth'].dtype}")


In [26]:
X_train, y_train, X_test, y_test = preprocessed_data

In [30]:
y_train

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,VehicleIntroDate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TransactionYear,TransactionMonthNum,TransactionQuarter,VehicleAge,PremiumToSumInsuredRatio,HasExcess,PowerToCapacityRatio,HighRiskProvince,IsCommercialVehicle
583200,224164,13084,2015-07-01,False,,Individual,Mr,English,Current account,Not specified,Not specified,South Africa,Gauteng,2000,Johannesburg,Johannesburg,Mobility - Motor,60058419.0,Passenger Vehicle,2012,TOYOTA,QUANTUM 2.7 SESFIKILE 16s,4.0,2694.0,111.0,B/S,4.0,4/2012,Yes,No,0,240000.00,Monthly,737.8418,Mobility - Taxi with value more than R100 000 ...,Own Damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,647.229649,2015,7,3,12,0.003074,1,0.041203,1,0
461400,147093,13040,2015-06-01,False,,Individual,Mr,English,Current account,Not specified,Not specified,South Africa,Gauteng,1466,Rand East,Rand East,Mobility - Motor,60058419.0,Passenger Vehicle,2013,TOYOTA,QUANTUM 2.7 SESFIKILE 16s,4.0,2694.0,111.0,B/S,4.0,4/2012,Yes,No,0,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,21.929825,2015,6,2,11,2499.750025,1,0.041203,1,0
13638,198058,12764,2015-07-01,False,,Individual,Mr,English,Current account,Not specified,Not specified,South Africa,Gauteng,199,Transvaal (Pretoria),Pretoria,Mobility - Motor,60058419.0,Passenger Vehicle,2014,TOYOTA,QUANTUM 2.7 SESFIKILE 16s,4.0,2694.0,111.0,B/S,4.0,4/2012,Yes,Yes,282100,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,21.929825,2015,7,3,10,2499.750025,1,0.041203,1,0
570009,72006,5861,2015-03-01,False,,Individual,Mr,English,Current account,Not specified,Not specified,South Africa,North West,308,Transvaal (all except Pretoria),Transvaal North West,Mobility - Motor,44004770.0,Medium Commercial,2013,MERCEDES-BENZ,SPRINTER 519 CDi F/C P/V,6.0,2987.0,140.0,P/V,4.0,12/2011,Yes,Yes,540000,0.01,Monthly,25.0000,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,21.929825,2015,3,1,11,2499.750025,1,0.046870,0,0
480399,118928,7251,2015-03-01,False,,Individual,Mr,English,Current account,Not specified,Not specified,South Africa,KwaZulu-Natal,4359,Natal,North Coast,Mobility - Motor,60058418.0,Passenger Vehicle,2011,TOYOTA,QUANTUM 2.7 SESFIKILE 15s,4.0,2694.0,111.0,B/S,4.0,10/2010,Yes,Yes,0,158000.00,Monthly,932.2486,Mobility - Taxi with value more than R100 000 ...,Own Damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,817.761930,2015,3,1,13,0.005900,1,0.041203,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472516,196711,10160,2015-04-01,False,,Individual,Mr,English,Savings account,Not specified,Not specified,South Africa,Gauteng,2000,Johannesburg,Johannesburg,Mobility - Motor,60058418.0,Passenger Vehicle,2010,TOYOTA,QUANTUM 2.7 SESFIKILE 15s,4.0,2694.0,111.0,B/S,4.0,10/2010,Yes,No,0,200000.00,Monthly,645.6115,Mobility - Taxi with value more than R100 000 ...,Own Damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,566.325877,2015,4,2,14,0.0032

In [33]:
# Check for any remaining datetime columns that might be causing the error
print("Checking for datetime columns in X_train:")
datetime_columns = X_train.select_dtypes(include=['datetime64']).columns.tolist()
print(f"Datetime columns found: {datetime_columns}")

print("\nChecking for datetime columns in X_test:")
datetime_columns_test = y_train.select_dtypes(include=['datetime64']).columns.tolist()
print(f"Datetime columns found: {datetime_columns_test}")

# Convert any remaining datetime columns to string format
if datetime_columns:
    for col in datetime_columns:
        X_train[col] = X_train[col].dt.strftime('%Y-%m-%d')
        print(f"Converted {col} in X_train to string format")

if datetime_columns_test:
    for col in datetime_columns_test:
        y_train[col] = y_train[col].dt.strftime('%Y-%m-%d')
        print(f"Converted {col} in y_train to string format")

# Verify all columns are now non-datetime
print("\nVerifying data types after conversion:")
print("X_train dtypes:")
print(X_train.dtypes)
print("\nX_test dtypes:")
print(y_train.dtypes)

# Now try training the models again
print("\nAttempting to train models again...")
models = modeling.train_models(X_train, X_test)


Checking for datetime columns in X_train:
Datetime columns found: []

Checking for datetime columns in X_test:
Datetime columns found: []

Verifying data types after conversion:
X_train dtypes:
UnderwrittenCoverID           int64
PolicyID                      int64
TransactionMonth             object
IsVATRegistered                bool
Citizenship                  object
LegalType                    object
Title                        object
Language                     object
AccountType                  object
MaritalStatus                object
Gender                       object
Country                      object
Province                     object
PostalCode                    int64
MainCrestaZone               object
SubCrestaZone                object
ItemType                     object
mmcode                      float64
VehicleType                  object
RegistrationYear              int64
make                         object
Model                        object
Cylinders     

In [35]:
evaluate = modeling.evaluate_models(models, y_train, y_test)

Evaluating models...
linear_regression: RMSE=0.0000, R²=1.0000, MAE=0.0000
random_forest: RMSE=0.0000, R²=1.0000, MAE=0.0000
xgboost: RMSE=0.0000, R²=1.0000, MAE=0.0000

Best model: linear_regression


In [36]:
evaluate.best_model

AttributeError: 'dict' object has no attribute 'best_model'

In [ ]:
modeling.analyze_feature_importance()